<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/hungry_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
# Installing the necessary packages
!pip install accelerate
!pip install bitsandbytes
!pip install SentencePiece
!pip install evaluate
!pip install bert_score
!pip install transformers
!apt install libgraphviz-dev
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [4]:
!nvidia-smi

Sun Nov 19 13:01:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# Import necessary modules
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer

In [6]:
load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [7]:
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-ldpeshx8
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-ldpeshx8
  Resolved https://github.com/deepset-ai/haystack.git to commit 9b11462bf886ef0c39e282d5378503d4ede1a7ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s 

In [8]:
from haystack.nodes.base import BaseComponent

class Query_Rewriter(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
      self.model = model
      self.tokenizer = tokenizer

    def run(self, query: str):

      prompt=f"""Generate some follow up questions I need to better answer your query
                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Answer:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  Question: {query}
                  Answer:"""
      input = self.tokenizer(prompt, return_tensors='pt')
      input_ids = input["input_ids"].to("cuda")
      temperature=0.7
      with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            temperature=temperature,
            top_p = 1.0,
            do_sample=True,
            return_dict_in_generate=True,
            max_new_tokens=200,
          )

      s = generation_output.sequences[0][len(input_ids[0]):]

      output = self.tokenizer.decode(s)
      return {"query":query, "prompt":prompt, "output":output}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [9]:
# Create Custom Query_Rewriter
query_rewriter = Query_Rewriter(model_l, tokenizer_l)

In [10]:
from haystack.pipelines import Pipeline

# Create a Haystack pipeline
p = Pipeline()
p.add_node(component=query_rewriter, name="QueryRewriter", inputs=["Query"])

In [21]:
indian_query = "Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?"

In [22]:
indian_result = p.run(query = indian_query)

In [23]:
print(f'''
  query: {indian_result["query"]}
  output: {indian_result["output"]}
  ''')


  query: Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?
  output: 
                  1. How spicy do you prefer your Indian food to be?
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. How many people are in your group?
                  5. What time are you planning to have your meal?</s>  
  


In [37]:
updated_indian_query = {
    "question":indian_result["query"],
    "clarifier": indian_result["output"],
    "additional_user_criteria":
      """
      1. NIL
      2. NIL
      3. Cheap
      4. NIL
      """
}

In [25]:
korean_query = "I want to buy bubble tea and then have Korean fried chicken. Can you give me some suggestions?"

In [26]:
korean_result = p.run(query = korean_query)

In [27]:
print(f'''
  query: {korean_result["query"]}
  output: {korean_result["output"]}
  ''')


  query: I want to buy bubble tea and then have Korean fried chicken. Can you give me some suggestions?
  output: 
                  1. What type of bubble tea are you looking for? e.g. milk tea, fruit tea, etc.
                  2. Are you looking for a specific type of Korean fried chicken? e.g. soy garlic, spicy, etc.
                  3. Do you have a preference for a specific brand or type of restaurant?
                  4. Are you looking for a quick and easy grab-and-go option or a sit-down meal?
                  5. How close do you want to be to your bubble tea and Korean fried chicken?</s>  
  


In [29]:
updated_korean_query = {
    "question":korean_result["query"],
    "clarifier": korean_result["output"],
    "additional_user_criteria":
      """
      1. NIL
      2. NIL
      3. NIL
      4. Sit-down
      5. Should be in the same mall
      """
}

In [157]:
class Annotate_Needs(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
      self.model = model
      self.tokenizer = tokenizer

    def run(self, query: dict):
      print(f'Summarizing user needs.... using query:{query}')

      prompt=f"""Generate the user's query requirements annotated with the requirement labels. Here are the categories:
                    Price, Opening, Location, Cuisine, Dietary Restrictions, Ratings, Reservations, Landmark, General (Atmosphere, idea-related needs)

                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Clarifying Questions:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  Additional User Criteria:
                  1. Sushi
                  2. Mostly just sushi
                  3. $$
                  4. Intimate with nice lighting, Inside Marina Bay Sands
                  5. 22:00 minimally
                  Answer:
                  Sushi[Cuisine], $$ [Price], Intimate with nice lighting [General-Atmosphere], Inside Marina Bay Sands [Location], 22:00 minimally [Opening]

                  ________________________________________________________________________________________________________
                  Question: {query["question"]}
                  Clarifying Questions: {query["clarifier"]}
                  Additional User Criteria:{query["additional_user_criteria"]}
                  Answer:"""
      input = self.tokenizer(prompt, return_tensors='pt')
      input_ids = input["input_ids"].to("cuda")
      temperature=0.7
      with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            temperature=temperature,
            top_p = 1.0,
            do_sample=True,
            return_dict_in_generate=True,
            max_new_tokens=200,
          )

      s = generation_output.sequences[0][len(input_ids[0]):]

      query["requirements"] = self.tokenizer.decode(s)
      return query, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [175]:
class Search_Term_Generator(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
      self.model = model
      self.tokenizer = tokenizer
      # TODO: Might need to change the temperature to be less random and favour more "safe" search terms
      # TODO: Might reduce the max tokens generated as well

    def run(self, query: dict):
      print(f'Generating Search Query.... with query:{query}')
      prompt=f"""Generate the search queries based on the example given below. Here are the categories and increasing importance (1-5), include more important terms more often in queries:
                    Price - 3, Opening - 4, Location - 5, Cuisine - 5, Dietary Restrictions - 5, Ratings - 2, Reservations - 2, Landmark - 2 , General - 1 (Atmosphere, idea-related needs)

                  Example:
                  Question:
                  Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Clarifying Questions:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  Additional User Input:
                  1. Sushi
                  2. Mostly just sushi
                  3. $$
                  4. Intimate with nice lighting, Inside Marina Bay Sands
                  5. 22:00 minimally
                  Requirements:
                  Sushi[Cuisine], $$ [Price], Intimate with nice lighting [General-Atmosphere], Inside Marina Bay Sands [Location], 22:00 minimally [Opening]
                  Answer:
                  Sushi Restaurant $$ in MBS, Marina Bay Sands Japanese food open till 2200, Romantic Japanese restaurant in Marina Bay Sands, Central Sushi open till late 30-40SGD

                  ________________________________________________________________________________________________________
                  Question:
                  {query["question"]}
                  Clarifying Questions:
                  {query["clarifier"]}
                  Additional User Input:
                  {query["additional_user_criteria"]}
                  Requirements:
                  {query["requirements"]}
                  Answer:
                  """
      input = self.tokenizer(prompt, return_tensors='pt')
      input_ids = input["input_ids"].to("cuda")
      temperature=0.3
      with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            temperature=temperature,
            top_p = 1.0,
            do_sample=True,
            return_dict_in_generate=True,
            max_new_tokens=100,
          )

      s = generation_output.sequences[0][len(input_ids[0]):]

      output = self.tokenizer.decode(s)
      return {"query":query["question"], "clarifier": query["clarifier"], "additional_user_criteria":query["additional_user_criteria"], "requirements":query["requirements"], "search_terms":output}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [176]:
annotater = Annotate_Needs(model_l, tokenizer_l)
search_terms_generator = Search_Term_Generator(model_l, tokenizer_l)

# Create a Haystack pipeline
p2 = Pipeline()
p2.add_node(component=annotater, name="Annotate_Needs", inputs=["Query"])
p2.add_node(component=search_terms_generator, name="Search_Term_Generator", inputs=["Annotate_Needs"])

In [177]:
result2 = p2.run(query = updated_indian_query)

Summarizing user needs.... using query:{'question': 'Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?', 'clarifier': '\n                  1. How spicy do you prefer your Indian food to be?\n                  2. Do you have a preference for specific dishes or would you like to try a variety of options?\n                  3. Are you looking for a specific price range for the food?\n                  4. How many people are in your group?\n                  5. What time are you planning to have your meal?</s>', 'additional_user_criteria': '\n      1. NIL\n      2. NIL\n      3. Cheap\n      4. NIL\n      '}
Generating Search Query.... with query:{'question': 'Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?', 'clarifier': '\n                  1. How spicy do you prefer your Indian food to be?\n                  2. Do you have a preference for specific dishes or would you like to tr

In [178]:
print(f'''
  query: {result2["query"]}
  clarifier: {result2["clarifier"]}
  additional_user_criteria: {result2["additional_user_criteria"]}
  requirements:{result2["requirements"]}
  search_terms: {result2["search_terms"]}
  ''')


  query: Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?
  clarifier: 
                  1. How spicy do you prefer your Indian food to be?
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. How many people are in your group?
                  5. What time are you planning to have your meal?</s>  
  additional_user_criteria: 
      1. NIL
      2. NIL
      3. Cheap
      4. NIL
        
  requirements:
                  Indian [Cuisine], $$ [Price], group of 10 people [General-Atmosphere], near Esplanade [Location], 19:00 [Opening]</s>
  search_terms: 
                  Indian Restaurant $$ near Esplanade suitable for group of 10, Cheap Indian food in Singapore, Indian food open till 1900 near Esplanade, Group Indian food near Esplanade, Indian restaurant near Esplanade 

## Archive

In [148]:
from haystack import BaseComponent

class CustomNodeA(BaseComponent):
    outgoing_edges = 1

    def run(self, query: dict):
        # Append some text to the input
        print("A",query)
        query["text"] += " processed by NodeA"
        return query, "output_1"

    def run_batch(self, queries: list):
        # Insert code here to manipulate the input and produce an output dictionary
        input_data = []
        for query in queries:
            input_data.append(self.run(query))
        return input_data, "output_1"

class CustomNodeB(BaseComponent):
    outgoing_edges = 1

    def run(self, query: dict):
        # Append some more te
        print("B", query)
        query["text"] += " and then by NodeB"
        return query, "output_1"
    def run_batch(self, queries: list):
        # Insert code here to manipulate the input and produce an output dictionary
        output_2 = []
        for query in queries:
            output_2.append(self.run(query))
        return output_2, "output_1"

class NodeC(BaseComponent):
    outgoing_edges = 1
    def run(self, query:dict):
      print("C", query)
      query["hi"] = "hi"
      return query, "output_1"
    def run_batch(self, queries:list):
      return queries,"output_1"

In [151]:
from haystack import Pipeline

# Create instances of your custom nodes
node_a = CustomNodeA()
node_b = CustomNodeB()
node_c = NodeC()

# Create a Haystack pipeline and add the nodes
pipeline = Pipeline()
pipeline.add_node(component=node_a, name="NodeA", inputs=["Query"])
pipeline.add_node(component=node_b, name="NodeB", inputs=["NodeA"])
pipeline.add_node(component=node_c, name="NodeC", inputs=["NodeB"])


In [123]:
pipeline.draw("pipeline_retrieval.png")

In [152]:

# Prepare initial input and run the pipeline
initial_input= {"text": "Start"}
output = pipeline.run(query=initial_input)

print(output)

A {'text': 'Start'}
B {'text': 'Start processed by NodeA', 'root_node': 'Query', 'params': {}, 'query': {...}, 'node_id': 'NodeB'}
C {'text': 'Start processed by NodeA and then by NodeB', 'root_node': 'Query', 'params': {}, 'query': {...}, 'node_id': 'NodeC'}
{'text': 'Start processed by NodeA and then by NodeB', 'root_node': 'Query', 'params': {}, 'query': {...}, 'node_id': 'NodeC', 'hi': 'hi'}
